In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import matplotlib as mpl
from keras import Model
from sklearn.model_selection import train_test_split
import os
import cv2 as cv
import tqdm
from sklearn.preprocessing import StandardScaler

2025-05-08 18:43:01.043300: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-08 18:43:01.089623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-08 18:43:01.617267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# import kagglehub

# Download latest version
# path = kagglehub.dataset_download("rajkumarl/people-clothing-segmentation") #CC0: Public Domain
# path = kagglehub.dataset_download("agrigorev/clothing-dataset-full") #CC0: Public Domain
# path = kagglehub.dataset_download("ryanbadai/clothes-dataset") #MIT
# path = kagglehub.dataset_download("adisongoh/fashion-clothing-segmentation-dataset") #CC0: Public Domain
# path = kagglehub.dataset_download("abeerelmorshedy/fashion-clothes") #Unknown
# path = kagglehub.dataset_download("sourabhsingh03993493/clothessegmentation") #same as "adisongoh/fashion-clothing-segmentation-dataset"
# path = kagglehub.dataset_download("shreyanshverma27/new-data-fashion") #CC0: Public Domain
# path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-dataset") #MIT
# path = kagglehub.dataset_download("andhikawb/fashion-mnist-png") #MIT

# !mv /home/cat/.cache/kagglehub/datasets/andhikawb/fashion-mnist-png/versions/1 /home/cat/projects/CV-stuff/background_remover/datasets

In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="kp9qURyWHo7oCsHf7ngz")
# project = rf.workspace("umut-saydam-03gn7").project("clothescombined")
# version = project.version(3)
# dataset = version.download("coco-segmentation")

# dataset = version.download("sam2", "/home/cat/projects/CV-stuff/background_remover/sam2")

# from roboflow import Roboflow
# rf = Roboflow(api_key="kp9qURyWHo7oCsHf7ngz")
# project = rf.workspace("wep-7g80m").project("clothing-dcqa4")
# version = project.version(4)
# dataset = version.download("coco-segmentation")

# project = rf.workspace("fashion-7bqvr").project("coco-fpn0s")
# version = project.version(2)
# dataset = version.download("coco-segmentation")

# project = rf.workspace("i-style-you-daily").project("i-style-you-daily")
# version = project.version(1)
# dataset = version.download("coco-segmentation")
                
                          
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to I-Style-You-Daily-1 in coco-segmentation:: 100%|██████████| 251/251 [00:00<00:00, 9230.62it/s]


In [57]:
DATASET_PATH = "/home/cat/projects/CV-stuff/background_remover/datasets/clothessegmentation"
SET = DATASET_PATH + "/Test"
IMAGES_PATH = SET + "/Image/"
MASKS_PATH = SET + "/Mask/"

NEW_IMAGES_PATH = DATASET_PATH + "/Image/"
NEW_MASK_PATH = DATASET_PATH + "/Mask/"

In [ ]:
# transform dataset with images as masks

def transform(path: str, new_path: str):
    classes = [0, 17, 19, 20, 41, 47]
    if not os.path.exists(new_path): # if there is no exist, make the path
        os.makedirs(new_path)
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        if filename.endswith(".png"):
            image = cv.imread(file_path, cv.IMREAD_GRAYSCALE)
            # Apply thresholding to create a binary mask
            binary_mask = (image < 255).astype(np.uint8)  # Binary mask where 1 represents actual mask and 0 represents whitespace

            # Find bounding box of actual mask
            nonzero_indices = np.nonzero(binary_mask)
            min_row, min_col = np.min(nonzero_indices, axis=1)
            max_row, max_col = np.max(nonzero_indices, axis=1)

            # Crop the original mask using the bounding box
            image = image[min_row:max_row+1, min_col:max_col+1]
            clothes_mask = np.where((image > 60) & (image < 150), 1, 0).astype(np.uint8)
            mask_image = np.repeat(clothes_mask[:, :, np.newaxis], 3, 2)
            cv.imwrite(new_path + filename.split(".")[0]+".png", mask_image)
            # plt.imshow(image, cmap="gray")
        else:
            image = cv.imread(file_path, cv.IMREAD_COLOR)
            cv.imwrite(new_path + filename.split(".")[0]+".png", image)


In [ ]:
# transform(IMAGES_PATH, NEW_IMAGES_PATH)
# transform(MASKS_PATH, NEW_MASK_PATH)

In [31]:
# transform sam2 datasest with labels in coco annotation file file 
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
import numpy as np
from PIL import Image
import os


COCO_DATASET_PATH = "/home/cat/projects/CV-stuff/background_remover/datasets/COCO-2"
SET = COCO_DATASET_PATH + "/train"

NEW_MASK_PATH = COCO_DATASET_PATH + "/Mask/"
NEW_IMAGE_PATH = COCO_DATASET_PATH + "/Image/"


In [21]:
def transform_coco(path: str, new_mask_path: str, new_image_path: str):
    if not os.path.exists(new_mask_path): # if there is no exist, make the path
        os.makedirs(new_mask_path)
    if not os.path.exists(new_image_path): # if there is no exist, make the path
        os.makedirs(new_image_path)
# Load your COCO annotations file
    coco = COCO(path+"/_annotations.coco.json")

    # Loop through images
    for img_id in coco.getImgIds():
        img_info = coco.loadImgs(img_id)[0]
        height, width = img_info["height"], img_info["width"]
        # print(img_info["file_name"][:-4])
        anns = coco.loadAnns(coco.getAnnIds(imgIds=img_id))

        # Create empty mask
        mask = np.zeros((height, width), dtype=np.uint8)

        for i, ann in enumerate(anns, 1):
            # Flattened polygon points from ann["segmentation"][0]
            polygon = ann["segmentation"][0]

            # Convert to NumPy array and reshape to (N, 1, 2) for OpenCV
            pts = np.array(polygon, dtype=np.int32).reshape(-1, 1, 2)

            # Fill polygon with value 1
            cv.fillPoly(mask, [pts], color=1)
            # binary_mask = maskUtils.decode(rle)
            # mask[binary_mask[:, :, 0] > 0] = 1

        # Save mask
        mask_img = Image.fromarray(mask)
        mask_img.save(new_mask_path + img_info["file_name"][:-4] + ".png")
        png_image = Image.open(path+"/"+img_info["file_name"])
        png_image.save(new_image_path + img_info["file_name"][:-4] + ".png")


In [32]:
transform_coco(SET, NEW_MASK_PATH, NEW_IMAGE_PATH)

loading annotations into memory...
Done (t=0.37s)
creating index...
index created!


In [3]:
image = cv.imread("/home/cat/projects/CV-stuff/background_remover/datasets/clothing-dataset-full/results/mask/00a1b7a8-217d-45bf-93a8-86db0bdf9d9d.png", cv.IMREAD_GRAYSCALE)

In [13]:
np.unique(image)
image.shape
print(image[2000][200:500])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   5  11  18  24  37  8

In [35]:
DATASET_PATH = "/home/cat/projects/CV-stuff/background_remover/datasets/clothing-dataset-full/results/Image"
NEW_PATH = "/home/cat/projects/CV-stuff/background_remover/datasets/clothing-dataset-full/resized/Image"

In [38]:
# transform masks where mask is not strictly 0 and 1

def transform_to_binary(path: str, new_path: str):
    if not os.path.exists(new_path): # if there is no exist, make the path
        os.makedirs(new_path)
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        image = cv.imread(file_path, cv.IMREAD_COLOR)

        # binary_mask = np.where((image > 0), 1, 0).astype(np.uint8)
        # mask_image = np.repeat(binary_mask[:, :, np.newaxis], 3, 2)
        # Resize
        # Define the new width
        new_width = 640

        # Calculate the new height while maintaining the aspect ratio
        height, width = image.shape[:2]
        aspect_ratio = width / height
        new_height = int(new_width / aspect_ratio)
        image = cv.resize(image, (new_width, new_height), interpolation=cv.INTER_NEAREST)
        # print(os.path.join(new_path, filename), mask_image.shape, image.shape)
        cv.imwrite(os.path.join(new_path, filename), image)

transform_to_binary(DATASET_PATH, NEW_PATH)